**A partir del archivo .csv con datos del resultado de una encuesta de calidad en una aerolínea comercial, se obtienen los valores de las siguientes métricas operativas de la empresa.**

* ¿Cuántas personas consideran que el sistema de reservas online (Ease of Online booking) tiene un nivel de satisfacción igual o superior a 3 puntos? Para cada persona, indicad el Género (Gender) y la edad (Age)?
* ¿Cuántas observaciones se registran para el vuelo de mayor distancia (Flight Distance)? Para cada observación indicad tipo de cliente (Customer Type) y tipo de viaje (Type of Travel)?
* Filtrar las observaciones para el tipo de cliente (Customer Type) “Loyal Customer” y ordenar de manera descendente por la distancia de vuelo (Flight Distance).  Para cada observación, indicad el Género (Gender), el tipo de cliente (Customer Type y la distancia de vuelo (Flight Distance).
* Filtrar las observaciones para el nivel de satisfaccion (satisfaction) “neutral or dissatisfied” y sumar para cada observacion el retraso en la partida (Departure Delay in Minutes) más el retraso en la llegada (Arrival  Delay in Minutes). Mostrar la suma de los retrasos mencionados, el nivel de satisfaccion (satisfaction) , el tipo de cliente (Customer Type) y la distancia de vuelo (Flight Distance).
* Filtrar las observaciones para el nivel de satisfacción (satisfaction) “satisfied” y sumar para cada observación el retraso en la partida (Departure Delay in Minutes) más el retraso en la llegada (Arrival Delay in Minutes).  Mostrar la suma de los retrasos mencionados, el nivel de satisfacción (satisfacción), el tipo de cliente (Customer Type) y la distancia de vuelo (Flight Distance).  
* Indicar para los resultados de las métricas 4 y 5 si observa alguna correlacion entre el nivel de satisfaccion del cliente (satisfaction) y el tipo de cliente (Customer Type) en función de los retrasos experimentados. 


**Consideraciones generales:**

* Cada línea del archivo .csv contiene los resultados para una persona y corresponde a una observación. 
* El dataset se provee con la cabecera en la primera línea, antes de procesarlo con RDDs removed esta primera línea para hacer el proceso más simple.
* Los niveles de satisfacción se indican en una escala del 0 al 5. La escala se indica a continuación: 
  * Valor “0”: Muy insatisfecho
  * Valor “1”: Insatisfecho
  * Valor “2”: Algo Insatisfecho
  * Valor “3”: Ni satisfecho ni insatisfecho
  * Valor “4”: Algo satisfecho
  * Valor “5”: Muy Satisfecho


In [0]:
%fs
ls dbfs:/FileStore/tables/

path,name,size,modificationTime
dbfs:/FileStore/tables/encuesta_aerolinea-1.csv,encuesta_aerolinea-1.csv,11986211,1706643440000
dbfs:/FileStore/tables/encuesta_aerolinea.csv,encuesta_aerolinea.csv,11986211,1706643304000
dbfs:/FileStore/tables/encuesta_aerolinea_simplificado-1.csv,encuesta_aerolinea_simplificado-1.csv,11986211,1705509312000
dbfs:/FileStore/tables/encuesta_aerolinea_simplificado.csv,encuesta_aerolinea_simplificado.csv,11986211,1705509276000



Los RDD, o Resilient Distributed Datasets, son una abstracción para manejar y procesar grandes conjuntos de datos de manera distribuida y paralela en un entorno de cluster. Un RDD es una colección de elementos que están distribuidos a través de los nodos del cluster de Spark. Esto significa que los datos se dividen en varias partes que se procesan simultáneamente en diferentes nodos. Esta distribución es clave para lograr un procesamiento de datos de alta velocidad y eficiencia en entornos de Big Data.

Se puede pensar en un RDD como una colección de objetos, similar a una lista o un arreglo en lenguajes de programación como Python o Java. Sin embargo, a diferencia de estas estructuras de datos convencionales, los elementos en un RDD están distribuidos en un cluster y se procesan en paralelo.

En Spark, los RDD se manipulan mediante dos tipos de operaciones: transformaciones y acciones. Las transformaciones, como `map` y `filter`, crean un nuevo RDD a partir de uno existente. Las acciones, como `count` y `collect`, devuelven un resultado al programa o escriben datos en un sistema de almacenamiento. 

Aunque los RDD fueron la abstracción de datos primaria en las primeras versiones de Spark, las versiones más recientes introducen APIs de alto nivel como los DataFrames y los Datasets. Los RDD siguen siendo útiles para ciertos casos de uso, especialmente cuando se necesita un control fino sobre las operaciones de bajo nivel.

In [0]:
lines = sc.textFile('dbfs:/FileStore/tables/encuesta_aerolinea.csv')
df = spark.read.format("csv").option("inferSchema","true").option("header","true").load("dbfs:/FileStore/tables/encuesta_aerolinea.csv")
df_limited = df.limit(5)  # Limita a 5 registros
df_limited.display()

_c0,id,Gender,Customer Type,Age,Type of Travel,Class,Flight Distance,Inflight wifi service,Departure/Arrival time convenient,Ease of Online booking,Gate location,Food and drink,Online boarding,Seat comfort,Inflight entertainment,On-board service,Leg room service,Baggage handling,Checkin service,Inflight service,Cleanliness,Departure Delay in Minutes,Arrival Delay in Minutes,satisfaction
0,70172,Male,Loyal Customer,13,Personal Travel,Eco Plus,460,3,4,3,1,5,3,5,5,4,3,4,4,5,5,25,18,neutral or dissatisfied
1,5047,Male,disloyal Customer,25,Business travel,Business,235,3,2,3,3,1,3,1,1,1,5,3,1,4,1,1,6,neutral or dissatisfied
2,110028,Female,Loyal Customer,26,Business travel,Business,1142,2,2,2,2,5,5,5,5,4,3,4,4,4,5,0,0,satisfied
3,24026,Female,Loyal Customer,25,Business travel,Business,562,2,5,5,5,2,2,2,2,2,5,3,1,4,2,11,9,neutral or dissatisfied
4,119299,Male,Loyal Customer,61,Business travel,Business,214,3,3,3,3,4,5,5,3,3,4,4,3,3,3,0,0,satisfied


In [0]:
def parser(line):
    fields = line.split(',')
    id = float(fields[1])
    gender = fields[2]
    customer_type = fields[3]
    age = float(fields[4])
    type_of_travel = fields[5]
    class_ = fields[6]
    flight_distance = float(fields[7])
    wifi = float(fields[8])
    time_convenient = float(fields[9])
    booking = float(fields[10])
    gate = float(fields[11])
    food = float(fields[12])
    boarding = float(fields[13])
    comfort = float(fields[14])
    entertainment = float(fields[15])
    on_board_service = float(fields[16])
    room_service = float(fields[17])
    handling = float(fields[18])
    checking = float(fields[19])
    inflight = float(fields[20])
    cleanliness = float(fields[21])
    departures_delay = float(fields[22])
    arrivals_delay = float(fields[23])
    satisfaction = fields[24]

    return (gender, age, booking, flight_distance, customer_type, type_of_travel, satisfaction, departures_delay, arrivals_delay)

In [0]:
header = lines.first()
display(header)
pre_parsed = lines.filter(lambda row: row != header)
display(pre_parsed.first())

',id,Gender,Customer Type,Age,Type of Travel,Class,Flight Distance,Inflight wifi service,Departure/Arrival time convenient,Ease of Online booking,Gate location,Food and drink,Online boarding,Seat comfort,Inflight entertainment,On-board service,Leg room service,Baggage handling,Checkin service,Inflight service,Cleanliness,Departure Delay in Minutes,Arrival Delay in Minutes,satisfaction''0,70172,Male,Loyal Customer,13,Personal Travel,Eco Plus,460,3,4,3,1,5,3,5,5,4,3,4,4,5,5,25,18,neutral or dissatisfied'

In [0]:
parsed = pre_parsed.map(parser).persist() # Persiste el RDD en el storagee level MEMORY_AND_DISK por defecto.
parsed

Out[6]: PythonRDD[49] at RDD at PythonRDD.scala:58

**1. ¿Cuántas personas consideran que el sistema de reservas online (Ease of Online booking) tiene un nivel de satisfacción igual o superior a 3 puntos? Para cada persona, indicad el Género (Gender) y la edad (Age)?**

In [0]:
# 1.Filtrar el RDD parsed para el nivel de satisfacción de 3 puntos.
maxs = parsed.filter(lambda x: x[2] >=3 )
# 2.Utilizar el método map() para mapear los tres campos requeridos y colectar con collect().
results = maxs.map(lambda x: (x[0], x[1], x[2])).collect()
print (len(results))
# 3.El método collect() devuelve una lista de Python cuyos elementos corresponde a cada elemento del RDD. Indexar la lista para obtener los valores solicitados.

for person in results:
    print(f"La persona de género {person[0]} tiene una edad de {person[1]} años y considera que el sistema de booking online tiene un nivel de satisfacción {person[2]} puntos")

57871
La persona de género Male tiene una edad de 13.0 años y considera que el sistema de booking online tiene un nivel de satisfacción 3.0 puntos
La persona de género Male tiene una edad de 25.0 años y considera que el sistema de booking online tiene un nivel de satisfacción 3.0 puntos
La persona de género Female tiene una edad de 25.0 años y considera que el sistema de booking online tiene un nivel de satisfacción 5.0 puntos
La persona de género Male tiene una edad de 61.0 años y considera que el sistema de booking online tiene un nivel de satisfacción 3.0 puntos
La persona de género Female tiene una edad de 52.0 años y considera que el sistema de booking online tiene un nivel de satisfacción 4.0 puntos
La persona de género Male tiene una edad de 20.0 años y considera que el sistema de booking online tiene un nivel de satisfacción 3.0 puntos
La persona de género Female tiene una edad de 24.0 años y considera que el sistema de booking online tiene un nivel de satisfacción 5.0 puntos
L

**2. ¿Cuántas observaciones se registran para el vuelo de mayor distancia (Flight Distance)? Para cada observación indicad tipo de cliente (Customer Type) y tipo de viaje (Type of Travel)?**

In [0]:
# 1.Mapear el RDD parsed para obtener un tuple del tipo (key, value) donde el key corresponde al elemento flight_distance y value a un tuple con los elementos customer_type y type_of_travel.
f_distance = parsed.map(lambda x: (x[3],(x[4],x[5])))
# 2.Ordernar for key el RDD f_distance de manera descendente.
distance_ordered = f_distance.sortByKey(ascending=False)
# 3.Colectar en la lista max_distance el RDD distance_ordered.
max_distance = distance_ordered.collect()
# 4.La distancia de vuelo máxima se encuentra en el primer elemento de la lista max_distance.

print(f"La distancia de vuelo maxima es: {max_distance[0][0]} Kilometros")

# 5.Filtrar los elementos de distance_ordered donde el primer elemento de este RDD (flight_distance) corresponde el valor máximo obtenido de la lista max_distance y luego colectar los datos.
max_distance_ob = distance_ordered.filter(lambda x: x[0] == max_distance[0][0]).collect()

# 6.Extraer la cantidad de observaciones con el método len() de Python, el cual devuelve la cantidad de elementos de la lista max_distance_ob.
print(f"La cantidad de observaciones a la distancia maxima de {max_distance[0][0]} Kilometros corresponde a {len(max_distance_ob)} viajes.")

La distancia de vuelo maxima es: 4983.0 Kilometros
La cantidad de observaciones a la distancia maxima de 4983.0 Kilometros corresponde a 12 viajes.


**3. Filtrar las observaciones para el tipo de cliente (Customer Type) “Loyal Customer” y ordenar de manera descendente por la distancia de vuelo (Flight Distance).  Para cada observación, indicad el Género (Gender), el tipo de cliente (Customer Type y la distancia de vuelo (Flight Distance).**

In [0]:
# 1. Filtrar el RDD parsed para los elementos que contengan el string “Loyal Customer”
loyal_cust_observations = parsed.filter(lambda x: "Loyal Customer" in x[4])
# 2. Mapear a una función lambda que cree un tuple con los elementos flight_distance como key y los elementos gender y customer_type como value.
loyal_cust =loyal_cust_observations.map(lambda x: (x[3],(x[4],x[0])))
# 3. Ordenar por key el RDD resultante del paso anterior y colectar.
ordered_loyal_cust = loyal_cust.sortByKey(ascending=False).collect()

# 4. Utilizando un bucle for de Python extraer los elementos de la lista resultante del paso anterior.
for obs in ordered_loyal_cust:
    print(f"Género: {obs[1][1]}, Tipo de cliente: {obs[1][0]}, Distancia de vuelo: {obs[0]}")

Género: Female, Tipo de cliente: Loyal Customer, Distancia de vuelo: 4983.0
Género: Female, Tipo de cliente: Loyal Customer, Distancia de vuelo: 4983.0
Género: Female, Tipo de cliente: Loyal Customer, Distancia de vuelo: 4983.0
Género: Female, Tipo de cliente: Loyal Customer, Distancia de vuelo: 4983.0
Género: Male, Tipo de cliente: Loyal Customer, Distancia de vuelo: 4983.0
Género: Female, Tipo de cliente: Loyal Customer, Distancia de vuelo: 4983.0
Género: Female, Tipo de cliente: Loyal Customer, Distancia de vuelo: 4983.0
Género: Male, Tipo de cliente: Loyal Customer, Distancia de vuelo: 4983.0
Género: Female, Tipo de cliente: Loyal Customer, Distancia de vuelo: 4983.0
Género: Male, Tipo de cliente: Loyal Customer, Distancia de vuelo: 4983.0
Género: Female, Tipo de cliente: Loyal Customer, Distancia de vuelo: 4983.0
Género: Female, Tipo de cliente: Loyal Customer, Distancia de vuelo: 4983.0
Género: Female, Tipo de cliente: Loyal Customer, Distancia de vuelo: 4963.0
Género: Female, Ti


**4.Filtrar las observaciones para el nivel de satisfaccion (satisfaction) “neutral or dissatisfied” y sumar para cada observacion el retraso en la partida (Departure Delay in Minutes) más el retraso en la llegada (Arrival  Delay in Minutes). Mostrar la suma de los retrasos mencionados, el nivel de satisfaccion (satisfaction) , el tipo de cliente (Customer Type) y la distancia de vuelo (Flight Distance).**

In [0]:
# 1. Filtrar el RDD parsed para los elementos del campo satisfaction que contengan el string "neutral or dissatisfied".
neutral_satisf = parsed.filter(lambda x: "neutral or dissatisfied" in x[6])
# 2. Mapear al RDD resultante del paso anterior una función lambda que extraiga los elementos requeridos sumando los elementos correspondientes a departures_delay y arrivals_delay y colectar.
total_delay = neutral_satisf.map(lambda x: (x[4], x[6], x[3], x[7] + x[8])). collect()
# 3. Iterar la lista resultante del paso anterior con un bucle for extrayendo de cada elemento los campos requeridos.
for obs in total_delay:
    print(f"Tipo de cliente: {obs[0]}, Satisfaccion: {obs[1]}, Distancia de vuelo: {obs[2]}, Retraso total {obs[3]}")

Tipo de cliente: Loyal Customer, Satisfaccion: neutral or dissatisfied, Distancia de vuelo: 460.0, Retraso total 43.0
Tipo de cliente: disloyal Customer, Satisfaccion: neutral or dissatisfied, Distancia de vuelo: 235.0, Retraso total 7.0
Tipo de cliente: Loyal Customer, Satisfaccion: neutral or dissatisfied, Distancia de vuelo: 562.0, Retraso total 20.0
Tipo de cliente: Loyal Customer, Satisfaccion: neutral or dissatisfied, Distancia de vuelo: 1180.0, Retraso total 0.0
Tipo de cliente: Loyal Customer, Satisfaccion: neutral or dissatisfied, Distancia de vuelo: 1276.0, Retraso total 32.0
Tipo de cliente: Loyal Customer, Satisfaccion: neutral or dissatisfied, Distancia de vuelo: 853.0, Retraso total 0.0
Tipo de cliente: disloyal Customer, Satisfaccion: neutral or dissatisfied, Distancia de vuelo: 1061.0, Retraso total 0.0
Tipo de cliente: disloyal Customer, Satisfaccion: neutral or dissatisfied, Distancia de vuelo: 1182.0, Retraso total 0.0
Tipo de cliente: Loyal Customer, Satisfaccion: n


**5.Filtrar las observaciones para el nivel de satisfacción (satisfaction) “satisfied” y sumar para cada observación el retraso en la partida (Departure Delay in Minutes) más el retraso en la llegada (Arrival Delay in Minutes).  Mostrar la suma de los retrasos mencionados, el nivel de satisfacción (satisfacción), el tipo de cliente (Customer Type) y la distancia de vuelo (Flight Distance).**

In [0]:
# 1. Filtrar el RDD parsed para los elementos del campo satisfaction que contengan el string "dissatisfied".
satisfied_cust = parsed.filter(lambda x: "dissatisfied" not in x[6])
# 2. Mapear al RDD resultante del paso anterior una función lambda que extraiga los elementos requeridos sumando los elementos correspondientes a departures_delay y arrivals_delay y colectar.
sat_total_delay = satisfied_cust.map(lambda x: (x[4], x[6], x[3], x[7] + x[8])). collect()
# 3. Iterar la lista resultante del paso anterior con un bucle for extrayendo de cada elemento los campos requeridos.
for obs in sat_total_delay:
    print(f"Tipo de cliente: {obs[0]}, Satisfaccion: {obs[1]}, Distancia de vuelo: {obs[2]}, Retraso total {obs[3]}")

Tipo de cliente: Loyal Customer, Satisfaccion: satisfied, Distancia de vuelo: 1142.0, Retraso total 0.0
Tipo de cliente: Loyal Customer, Satisfaccion: satisfied, Distancia de vuelo: 214.0, Retraso total 0.0
Tipo de cliente: Loyal Customer, Satisfaccion: satisfied, Distancia de vuelo: 2035.0, Retraso total 4.0
Tipo de cliente: Loyal Customer, Satisfaccion: satisfied, Distancia de vuelo: 946.0, Retraso total 0.0
Tipo de cliente: Loyal Customer, Satisfaccion: satisfied, Distancia de vuelo: 2123.0, Retraso total 100.0
Tipo de cliente: Loyal Customer, Satisfaccion: satisfied, Distancia de vuelo: 2075.0, Retraso total 10.0
Tipo de cliente: Loyal Customer, Satisfaccion: satisfied, Distancia de vuelo: 2486.0, Retraso total 12.0
Tipo de cliente: disloyal Customer, Satisfaccion: satisfied, Distancia de vuelo: 452.0, Retraso total 98.0
Tipo de cliente: Loyal Customer, Satisfaccion: satisfied, Distancia de vuelo: 1561.0, Retraso total 0.0
Tipo de cliente: Loyal Customer, Satisfaccion: satisfied, D


**6. Indicar para los resultados de las métricas 4 y 5 si observa alguna correlacion entre el nivel de satisfaccion del cliente (satisfaction) y el tipo de cliente (Customer Type) en función de los retrasos experimentados.**

In [0]:
# Mapear el RDD para crear un par clave-valor (CustomerType, Satisfaction) como clave y TotalDelay como valor
mapped_rdd = parsed.map(lambda x: ((x[4], x[6]), (x[7] + x[8], 1)))

# Reducir por clave para sumar los retrasos y contar los registros
reduced_rdd = mapped_rdd.reduceByKey(lambda a, b: (a[0] + b[0], a[1] + b[1]))

# Calcular el promedio
avg_delay_rdd = reduced_rdd.mapValues(lambda x: x[0] / x[1])

# Recolectar y mostrar los resultados
for result in avg_delay_rdd.collect():
    print(f"Customer Type: {result[0][0]}, Satisfaction: {result[0][1]}, Average Delay: {result[1]}")


Customer Type: Loyal Customer, Satisfaction: neutral or dissatisfied, Average Delay: 33.99747690921379
Customer Type: disloyal Customer, Satisfaction: neutral or dissatisfied, Average Delay: 32.294154185934154
Customer Type: Loyal Customer, Satisfaction: satisfied, Average Delay: 25.178151136111317
Customer Type: disloyal Customer, Satisfaction: satisfied, Average Delay: 25.426981300089047


In [0]:
from pyspark.sql.functions import col

# Convertir RDD a DataFrame para facilitar el análisis
df = parsed.toDF(["Gender", "Age", "Booking", "FlightDistance", "CustomerType", "TypeOfTravel", "Satisfaction", "DepartureDelay", "ArrivalDelay"])

# Calcular el retraso total
df = df.withColumn("TotalDelay", col("DepartureDelay") + col("ArrivalDelay"))

# Calcular las métricas promedio de retraso por tipo de cliente y nivel de satisfacción
avg_delay_by_customer_type_and_satisfaction = df.groupBy("CustomerType", "Satisfaction").avg("TotalDelay").sort("CustomerType", "Satisfaction")

# Mostrar los resultados
avg_delay_by_customer_type_and_satisfaction.show()



+-----------------+--------------------+------------------+
|     CustomerType|        Satisfaction|   avg(TotalDelay)|
+-----------------+--------------------+------------------+
|   Loyal Customer|neutral or dissat...| 33.99747690921379|
|   Loyal Customer|           satisfied|25.178151136111317|
|disloyal Customer|neutral or dissat...|32.294154185934154|
|disloyal Customer|           satisfied|25.426981300089047|
+-----------------+--------------------+------------------+



**Conclusión:**

Los clientes que tuvieron un nivel de satisfacción como "neutral or dissatisfied" son los que tuvieron un promedio en el tiempo de retraso mayor a los 32 min, sin importar el Customer Type: Loyal or Disloyal

Por otra parte, los clientes que puntuaron con un nivel de satisfacción como "satisfied" son los que tuvieron un promedio en el tiempo de retraso alrededor a los 25 min, de igual manera, sin importar el Customer Type: Loyal or Disloyal.

Más allá del Customer Type: Loyal or Disloyal. Lo que hace que un cliente tenga un nivel de satisfacción como "satisfecho o insatisfecho" es el tiempo de retraso. A mayor tiempo de retraso es más probable que puntúe como insatisfecho.